# Triplets

Generate random triplets from pre-computed embedding vectors.

In [1]:
from livia.embedding import load_csv

# Load metadata and construct data frame from gzipped sample file
embeddings = load_csv("../data/sentence_embeddings/wm_sbert_title_subjects_256d.csv")

In [6]:
from livia.triplet import generate_triplets

NUMBER_OF_TRIPLETS = 500

random_triplets = generate_triplets(embeddings, method='clustering', n=NUMBER_OF_TRIPLETS)
random_triplets

500 triplets are generated. This may take a while ... 
Done!


[('620579', '493346', '550265'),
 ('564308', '346590', '163839'),
 ('234869', '367638', '37531'),
 ('453905', '423498', '446395'),
 ('1060583', '1060579', '545629'),
 ('1448566', '68531', '595563'),
 ('788551', '249988', '184876'),
 ('132562', '48202', '985179'),
 ('496697', '518060', '786574'),
 ('367638', '365424', '620914'),
 ('1189577', '688397', '312892'),
 ('480032', '480024', '94873'),
 ('335208', '223594', '94571'),
 ('130707', '130527', '159461'),
 ('43835', '1389635', '37446'),
 ('1015370', '43748', '422597'),
 ('59665', '180185', '412250'),
 ('566668', '566686', '87373'),
 ('445394', '128376', '555591'),
 ('675349', '692701', '134836'),
 ('310401', '416152', '697964'),
 ('676891', '676829', '1028352'),
 ('48127', '48572', '570167'),
 ('230859', '58942', '242863'),
 ('94191', '861634', '144617'),
 ('539602', '211711', '38621'),
 ('517005', '501038', '392859'),
 ('16142', '11432', '124677'),
 ('1040693', '117973', '433448'),
 ('580603', '184442', '179675'),
 ('678268', '108051

Triplets are just tuples of IDs. Now join relevant metadata.

In [7]:
import pandas as pd

df = pd.read_csv("../data/metadata/wien_museum.csv")
df = df.astype(str)

df

,id,url,inventoryNumber,title,artistsProducers,classifications,dates,districts,subjects,themes,multimedia_caption,multimedia_preview,multimedia_default,multimedia_creditline,multimedia_suggestedCitation
0,1902639,https://sammlung.wienmuseum.at/objekt/1902639/,310173,Handbeschriebener Zettel mit Angebot zur Nachb...,nan,Grafik- und Fotosammlung | Hand- und Druckschr...,2020,nan,"Handschrift, handgeschriebener Text","Handschrift, handgeschriebener Text",CC0,https://sammlung.wienmuseum.at/openapi-images/...,https://sammlung.wienmuseum.at/openapi-images/...,"Foto: Birgit und Peter Kainz, Wien Museum",Handbeschriebener Zettel mit Angebot zur Nachb...
1,1902640,https://sammlung.wienmuseum.at/objekt/1902640/,310174/1,Zetteldokumentation zu den Corona-Balkonkonzer...,nan,Gebrauchsgegenstände | Erinnerungsgegenstände ...,2020,17. Bezirk: Hernals,"Handschrift, handgeschriebener Text","Handschrift, handgeschriebener Text",CC0,https://sammlung.wienmuseum.at/openapi-images/...,https://sammlung.wienmuseum.at/openapi-images/...,"Foto: Birgit und Peter Kainz, Wien Museum",Zetteldokumentation zu den Corona-Balkonkonzer...
2,1902641,https://sammlung.wienmuseum.at/objekt/1902641/,310174/2,Zetteldokumentation zu den Corona-Balkonkonzer...,nan,Gebrauchsgegenstände | Erinnerungsgegenstände ...,2020,17. Bezirk: Hernals,"Handschrift, handgeschriebener Text","Handschrift, handgeschriebener Text",CC0,https://sammlung.wienmuseum.at/openapi-images/...,https://sammlung.wienmuseum.at/openapi-images/...,"Foto: Birgit und Peter Kainz, Wien Museum",Zetteldokumentation zu den Corona-Balkonkonzer...
3,1902642,https://sammlung.wienmuseum.at/objekt/1902642/,310174/3,Zetteldokumentation zu den Corona-Balkonkonzer...,nan,Grafik- und Fotosammlung | Gebrauchsgrafik,2020,nan,nan,nan,CC0,https://sammlung.wienmuseum.at/openapi-images/...,https://sammlung.wienmuseum.at/openapi-images/...,"Foto: Birgit und Peter Kainz, Wien Museum",Zetteldokumentation zu den Corona-Balkonkonzer...
4,1902643,https://sammlung.wienmuseum.at/objekt/1902643/,310174/4,Zetteldokumentation zu den Corona-Balkonkonzer...,nan,Gebrauchsgegenstände | Erinnerungsgegenstände ...,2020,17. Bezirk: Hernals,"Handschrift, handgeschriebener Text","Handschrift, handgeschriebener Text",CC0,https://sammlung.wienmuseum.at/openapi-images/...,https://sammlung.wienmuseum.at/openapi-images/...,"Foto: Birgit und Peter Kainz, Wien Museum",Zetteldokumentation zu den Corona-Balkonkonzer...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62586,1034409,https://sammlung.wienmuseum.at/objekt/1034409/,MV 94993,Helmbuschträger,Unbekannt,Archäologie | Eisen | Waffen,"1,280",nan,nan,nan,"CC BY 4.0, Foto: Birgit und Peter Kainz, Wien ...",https://sammlung.wienmuseum.at/openapi-images/...,https://sammlung.wienmuseum.at/openapi-images/...,"Foto: Birgit und Peter Kainz, Wien Museum","Unbekannt, Helmbuschträger, ca. 1–100 (typolog..."
62587,1033920,https://sammlung.wienmuseum.at/objekt/1033920/,MV 47556,Norisch-pannonische Flügelfibel Almgren 238,Unbekannt,Archäologie | Bronze (Buntmetalllegierung) | T...,1,nan,Accessoires,Accessoires,"CC BY 4.0, Foto: Birgit und Peter Kainz, Wien ...",https://sammlung.wienmuseum.at/openapi-images/...,https://sammlung.wienmuseum.at/openapi-images/...,"Foto: Birgit und Peter Kainz, Wien Museum","Unbekannt, Norisch-pannonische Flügelfibel Alm..."
62588,1033912,https://sammlung.wienmuseum.at/objekt/1033912/,MV 47568,Eingliedrige Spiralfibel,Unbekannt,Archäologie | Bronze (Buntmetalllegierung) | T...,1,nan,Accessoires,Accessoires,"CC BY 4.0, Foto: Birgit und Peter Kainz, Wien ...",https://sammlung.wienmuseum.at/openapi-images/...,https://sammlung.wienmuseum.at/openapi-images/...,"Foto: Birgit und Peter Kainz, Wien Museum","Unbekannt, Eingliedrige Spiralfibel, ca. 1–100..."
62589,910481,https://sammlung.wienmuseum.at/objekt/910481/,MV 25169/1123,Nauheimer Fibel Feugère 5 oder Fibel vom Typ J...,Unbekannt,Archäologie | Bronze (Buntmetalllegierung) | T...,-50,nan,Accessoires,Accessoires,"CC 

In [8]:
import json

triplets_with_metadata = []

def meta(row):
  return {
    'id': row['id'].values[0],
    'title': row['title'].values[0],
    'url': row['url'].values[0],
    'image': row['multimedia_default'].values[0]
  }

for triplet in random_triplets:
  a, b, c = triplet

  anchor = df.loc[df['id'] == a]
  similar = df.loc[df['id'] == b]
  dissimilar = df.loc[df['id'] == c]
  
  triplets_with_metadata.append({
    'anchor': meta(anchor),
    'similar': meta(similar),
    'dissimilar': meta(dissimilar)
  })

with open('triplets.json', 'w') as outfile:
  json.dump(triplets_with_metadata, outfile)